# Etki Tahminlerinin Çürütülmesi

Etki çürütmeleri iki türlüdür: negatif kontrol ve duyarlılık analizi.

# 1.Negatif kontrole dayalı çürütmeler

İlk tür çürütme testleri, herhangi bir iyi tahmin prosedürünün karşılaması gereken gerekli koşullardır. Bunlara negatif kontroller de denir. Bir tahminci çürütme testinde başarısız olursa (p değeri <0,05), bu tahmincide bir sorun olduğu anlamına gelir.

Negatif kontrol çürütme testleri şunlardan birine dayanır:

* **Değişmez dönüşümler (invariant transformations)**: Tahmini değiştirmemesi gereken verilerdeki değişiklikler. Orijinal veriler ile değiştirilmiş veriler arasında sonucu önemli ölçüde değişen herhangi bir tahminci testi geçemez. Örnekler: veri alt kümesi ve rastgele ortak neden çürütmeleridir.

* **Geçersiz kılan dönüşümler (nullifying transformations)**: Veriler değiştikten sonra, nedensel gerçek tahmin sıfırdır. Yeni verilerde sonucu sıfırdan önemli ölçüde değişen herhangi bir tahminci testi geçemez. Örnekler: plasebo tedavisi ve sahte sonuç (dummy outcome) çürütmeleridir.

# 2.Duyarlılık analizine dayalı çürütmeler

İkinci tür ise, gözlemlenmeyen karıştırıcı faktör bulunmaması (no unobserved confounding) gibi varsayımların ihlali durumunda elde edilen tahminin sağlamlığını test eden duyarlılık testleridir.

## 2.1.Plasebo Tedavisini Çürüten (Placebo Treatment Refuter)

Plasebo Tedavisi çürütmesi şunu sorar: Gerçek tedavi değişkenini bağımsız bir rastgele değişkenle değiştirdiğimizde tahmini nedensel etkiye ne olur? (İpucu: etki sıfıra gitmelidir)

```python
res_placebo=model.refute_estimate(identified_estimand, estimate,
       method_name="placebo_treatment_refuter", show_progress_bar=True, placebo_type="permute")
print(res_placebo)
```

## 2.2.Sahte Sonuç Çürütücüsü (Dummy Outcome Refuter)

Sahte sonuç çürütücü testleri: Gerçek sonuç değişkenini bağımsız bir rastgele değişkenle değiştirdiğimizde tahmini nedensel etkiye ne olur? (İpucu: Etki sıfıra gitmelidir) Ek olarak, testin bir uzantısı, nedensel etkinin sıfıra gitmesi gerekmeyen herhangi bir simüle edilmiş sonucu da kontrol edebilir: Verilen veri kümesine en yakın bilinen bir veri oluşturma sürecine dayalı simüle edilmiş bir sonuçla sonucu değiştirdiğimizde tahmini nedensel etkiye ne olur? (İpucu: Veri oluşturma sürecindeki etki parametresiyle eşleşmelidir)

Sıfır nedensel etki testi:

```python
ref = model.refute_estimate(identified_estimand,
                          causal_estimate,
                          method_name="dummy_outcome_refuter"
                          )
print(ref[0])
```

Sıfır olmayan nedensel etkinin test edilmesi:

```python
coefficients = np.array([1,2])
bias = 3
def linear_gen(df):
    y_new = np.dot(df[['W0','W1']].values,coefficients) + 3
    return y_new
```

```python
ref = model.refute_estimate(identified_estimand,
                          causal_estimate,
                          method_name="dummy_outcome_refuter",
                          outcome_function=linear_gen
                          )
print(ref[0])
```

## 2.3.Rastgele Ortak Neden Çürütücü (Random Common Cause Refuter)

Rastgele Ortak Neden çürütücüsü şunları kontrol eder: Veri setine ortak neden olarak bağımsız bir rastgele değişken ekledikten sonra tahmin yöntemi tahminini değiştirir mi? (İpucu: Değiştirmemelidir)

```python
res_random=model.refute_estimate(identified_estimand, estimate, method_name="random_common_cause", show_progress_bar=True)
print(res_random)
```

## 2.4.Veri Alt Örneklem Çürütücüsü (Data Subsample Refuter)

Bu test şunu soruyor: Verilen veri setini rastgele seçilmiş bir alt kümeyle değiştirdiğimizde tahmini etki önemli ölçüde değişir mi? (İpucu: değişmemeli)

```python
res_subset=model.refute_estimate(identified_estimand, estimate,
       method_name="data_subset_refuter", show_progress_bar=True, subset_fraction=0.9)
print(res_subset)
```

## 2.5.Duyarlılık Analizi

Duyarlılık analizi, veri setine tedavi ve sonuçla ilişkili ek bir ortak neden (karıştırıcı) eklediğimizde etki tahmininin ne kadar duyarlı olduğunu test eder? (İpucu: Çok hassas olmamalıdır)

### 2.5.1.Simülasyon tabanlı duyarlılık analizi

Bu çürütme bir p-değeri döndürmez. Bunun yerine, tanımlayıcı varsayımlar (identify_effect'te kullanılır) geçerli değilse tahminin ne kadar hızlı değiştiğine dair bir duyarlılık testi sağlar. Özellikle, arka kapı varsayımının ihlaline duyarlılığı kontrol eder: tüm yaygın nedenlerin gözlemlendiği.

Bunu yapmak için, tedavi ve sonuç arasında ek bir ortak nedene sahip yeni bir veri kümesi oluşturur. Ortak nedenin etkisini yakalamak için, yöntem girdi olarak ortak nedenin tedavi ve sonuç üzerindeki etkisinin gücünü alır. Ortak nedenin etkilerine ilişkin bu girdilere dayanarak, tedavi ve sonuç değerlerini değiştirir ve ardından tahmin ediciyi yeniden çalıştırır. Beklenen, yeni tahminin gözlemlenmeyen ortak nedenin küçük bir etkisiyle büyük ölçüde değişmemesi ve gözlemlenmeyen herhangi bir karıştırıcıya karşı sağlamlık göstermesidir.

Bu prosedürü yorumlamanın bir diğer eşdeğer yolu, girdi verilerinde zaten gözlemlenmemiş karıştırıcının mevcut olduğunu varsaymaktır. Tedavi ve sonuç değerlerindeki değişiklik, orijinal verilerde mevcut olan gözlemlenmemiş ortak nedenin etkisini ortadan kaldırır. Daha sonra tahminciyi bu değiştirilmiş veriler üzerinde yeniden çalıştırmak doğru tanımlanmış tahmini sağlar ve gözlemlenmemiş ortak nedenin etkisinin bazı sınırlı değerleri için yeni tahmin ile orijinal tahmin arasındaki farkın çok yüksek olmadığını umuyoruz.

**Alan bilgisinin önemi**: Bu test, gözlemlenmeyen karıştırıcının etkisinin makul girdi değerlerini belirlemek için alan bilgisini gerektirir. İlk olarak, karıştırıcının tedavi ve sonuç üzerindeki etkisinin tek bir değeri için sonucu gösteriyoruz.

```python
res_unobserved=model.refute_estimate(identified_estimand, estimate, method_name="add_unobserved_common_cause",
                                    confounders_effect_on_treatment="binary_flip", confounders_effect_on_outcome="linear",
                                   effect_strength_on_treatment=0.01, effect_strength_on_outcome=0.02)
print(res_unobserved)
```

Gözlemlenemeyen karıştırıcının etkisi arttıkça eğilimi incelemek genellikle daha faydalıdır. Bunun için, varsayılan karıştırıcıların etkilerinin bir dizisini sağlayabiliriz. Çıktı, farklı gözlemlenemeyen karıştırıcılar altında tahmini etkilerin (min, maks) aralığıdır.

```python
res_unobserved_range=model.refute_estimate(identified_estimand, estimate, method_name="add_unobserved_common_cause",
                                    confounders_effect_on_treatment="binary_flip", confounders_effect_on_outcome="linear",
                                   effect_strength_on_treatment=np.array([0.001, 0.005, 0.01, 0.02]), effect_strength_on_outcome=0.01)
print(res_unobserved_range)
```

Yukarıdaki grafik, tedavi üzerindeki varsayılan karıştırıcı etki arttıkça tahminin nasıl azaldığını göstermektedir. Alan bilgisine göre, tedavi üzerindeki maksimum olası karıştırıcı etkiyi bilebiliriz. Etkinin sıfırın ötesine geçmediğini gördüğümüzden, tedavi v0'ın nedensel etkisinin pozitif olduğu sonucuna güvenle varabiliriz.

Ayrıca, hem tedavi hem de sonuç üzerindeki karıştırıcı etkiyi değiştirebiliriz. Bir ısı haritası elde ederiz

```python
res_unobserved_range=model.refute_estimate(identified_estimand, estimate, method_name="add_unobserved_common_cause",
                                          confounders_effect_on_treatment="binary_flip", confounders_effect_on_outcome="linear",
                                          effect_strength_on_treatment=[0.001, 0.005, 0.01, 0.02],
                                          effect_strength_on_outcome=[0.001, 0.005, 0.01,0.02])
print(res_unobserved_range)
```

Son olarak, DoWhy etki gücü (effect strength) parametrelerinin otomatik olarak seçilmesini destekler. Bu, gözlemlenmeyen karıştırıcının tedavi veya sonuç üzerindeki etkisinin gözlemlenen herhangi bir karıştırıcıdan daha güçlü olamayacağı varsayımına dayanır. Yani, en azından en alakalı karıştırıcı için veri topladık. Eğer durum buysa, effect_strength_on_treatment ve effect_strength_on_outcome aralığını gözlemlenen karıştırıcıların etki gücüyle sınırlayabiliriz. Gözlemlenmeyen karıştırıcının etki gücünün gözlemlenen en yüksek kadar mı yoksa onun bir kesri kadar mı olması gerektiğini belirten ek bir isteğe bağlı parametre vardır. İsteğe bağlı effect_fraction_on_treatment ve effect_fraction_on_outcome parametrelerini kullanarak bunu ayarlayabilirsiniz. Varsayılan olarak, bu iki parametre 1'dir.

```python
res_unobserved_auto = model.refute_estimate(identified_estimand, estimate, method_name="add_unobserved_common_cause",
                                          confounders_effect_on_treatment="binary_flip", confounders_effect_on_outcome="linear")
print(res_unobserved_auto)
```

### 2.5.2.Doğrusal tahmin ediciler için kısmi-R2 tabanlı duyarlılık analizi

Bu gelişmiş bir çürütme testidir. Ayrıntılar Regresyon Modelleri için Duyarlılık Analizi not defterindedir.


### 2.5.3.Doğrusal olmayan tahmin ediciler için Reisz tahmin edicisine dayalı duyarlılık analizi

Bu, doğrusal olmayan tahmin ediciler için gelişmiş bir çürütme testidir. Ayrıntılar "Parametrik olmayan nedensel tahmin ediciler için duyarlılık analizi" not defterindedir.